In [43]:
import tkinter as tk
import random

# Constants
WIDTH = 400
HEIGHT = 400
SIZE = 20
MARGIN = 10
DELAYS = {"easy": 200, "medium": 150, "hard": 100}

# Initialize the game
window = tk.Tk()
window.title("Snake Game")
canvas = tk.Canvas(window, width=WIDTH + 2 * MARGIN, height=HEIGHT + 2 * MARGIN, bg="black")
canvas.pack()

# Set up the initial snake and food
snake = [(WIDTH // 2, HEIGHT // 2)]
direction = "Right"
score = 0
difficulty = "medium"  # Default difficulty
delay = DELAYS[difficulty]
paused = False

food_x = random.randint(0, (WIDTH - SIZE) // SIZE) * SIZE
food_y = random.randint(0, (HEIGHT - SIZE) // SIZE) * SIZE
food = canvas.create_oval(
    MARGIN + food_x, MARGIN + food_y, MARGIN + food_x + SIZE, MARGIN + food_y + SIZE, fill="red"
)

# Display the score
score_label = tk.Label(window, text="Score: 0", fg="white", bg="black")
score_label.pack()

# Display the difficulty options
difficulty_frame = tk.Frame(window)
difficulty_frame.pack()

difficulty_label = tk.Label(difficulty_frame, text="Difficulty:", fg="white", bg="black")
difficulty_label.pack(side=tk.LEFT)


def set_difficulty(new_difficulty):
    global difficulty, delay
    difficulty = new_difficulty
    delay = DELAYS[difficulty]


easy_button = tk.Button(difficulty_frame, text="Easy", command=lambda: set_difficulty("easy"))
easy_button.pack(side=tk.LEFT)

medium_button = tk.Button(difficulty_frame, text="Medium", command=lambda: set_difficulty("medium"))
medium_button.pack(side=tk.LEFT)

hard_button = tk.Button(difficulty_frame, text="Hard", command=lambda: set_difficulty("hard"))
hard_button.pack(side=tk.LEFT)

# Display the score history
score_history = []
score_history_label = tk.Label(window, text="Score History:", fg="white", bg="black")
score_history_label.pack()

score_history_text = tk.Text(window, height=5, width=20, fg="white", bg="black")
score_history_text.pack()


# Function to move the snake
def move():
    global direction, score, food_x, food_y, food

    if not paused:
        head_x, head_y = snake[0]

        if direction == "Right":
            head_x += SIZE
        elif direction == "Left":
            head_x -= SIZE
        elif direction == "Up":
            head_y -= SIZE
        elif direction == "Down":
            head_y += SIZE

        # Check if the snake hits the wall
        if (
            head_x < 0
            or head_x >= WIDTH
            or head_y < 0
            or head_y >= HEIGHT
        ):
            game_over()
            return

        # Check if the snake eats the food
        if head_x == food_x and head_y == food_y:
            score += 1
            score_label.config(text="Score: {}".format(score))
            canvas.delete(food)

            food_x = random.randint(0, (WIDTH - SIZE) // SIZE) * SIZE
            food_y = random.randint(0, (HEIGHT - SIZE) // SIZE) * SIZE
            food = canvas.create_oval(
                MARGIN + food_x,
                MARGIN + food_y,
                MARGIN + food_x + SIZE,
                MARGIN + food_y + SIZE,
                fill="red",
            )
        else:
            # Remove the tail of the snake
            canvas.delete("snake")
            snake.pop()

        # Update the snake's position
        snake.insert(0, (head_x, head_y))

        # Draw the snake
        canvas.delete("snake")
        for segment in snake:
            x, y = segment
            canvas.create_rectangle(
                MARGIN + x,
                MARGIN + y,
                MARGIN + x + SIZE,
                MARGIN + y + SIZE,
                fill="green",
                tags="snake",
            )

        # Check if the snake hits itself
        if len(snake) != len(set(snake)):
            game_over()
            return

    # Schedule the next move
    window.after(delay, move)


# Function to handle key events
def handle_key(event):
    global direction, paused

    if event.keysym == "Right" and direction != "Left":
        direction = "Right"
    elif event.keysym == "Left" and direction != "Right":
        direction = "Left"
    elif event.keysym == "Up" and direction != "Down":
        direction = "Up"
    elif event.keysym == "Down" and direction != "Up":
        direction = "Down"
    elif event.keysym == "p":
        paused = not paused
    elif event.keysym == "r":
        restart_game()


# Function to end the game
def game_over():
    canvas.delete("all")
    canvas.create_text(
        (WIDTH + 2 * MARGIN) // 2,
        (HEIGHT + 2 * MARGIN) // 2,
        text="Game Over",
        font=("Arial", 20),
        fill="red",
    )
    score_label.config(text="Final Score: {}".format(score))
    score_history.append(score)
    update_score_history()


# Function to update the score history
def update_score_history():
    score_history_text.delete("1.0", tk.END)
    for index, score in enumerate(score_history):
        score_history_text.insert(tk.END, f"Game {index + 1}: {score}\n")
        score_history_text.tag_add(f"score_{index}", f"{index + 1}.0", f"{index + 1}.end")
        score_history_text.tag_bind(f"score_{index}", "<Button-1>", lambda event, idx=index: delete_score(idx))


# Function to delete a score from the history
def delete_score(index):
    score_history.pop(index)
    update_score_history()


# Function to restart the game
def restart_game():
    global snake, direction, score, food_x, food_y, food, paused
    snake = [(WIDTH // 2, HEIGHT // 2)]
    direction = "Right"
    score = 0
    delay = DELAYS[difficulty]
    paused = False

    canvas.delete("all")
    canvas.delete("snake")
    canvas.delete(food)

    food_x = random.randint(0, (WIDTH - SIZE) // SIZE) * SIZE
    food_y = random.randint(0, (HEIGHT - SIZE) // SIZE) * SIZE
    food = canvas.create_oval(
        MARGIN + food_x,
        MARGIN + food_y,
        MARGIN + food_x + SIZE,
        MARGIN + food_y + SIZE,
        fill="red",
    )

    score_label.config(text="Score: {}".format(score))
    update_score_history()

    move()


# Bind key events to the window
window.bind("<KeyPress>", handle_key)

# Start the game
def start_game():
    global snake, direction, score, food_x, food_y, food, paused
    snake = [(WIDTH // 2, HEIGHT // 2)]
    direction = "Right"
    score = 0
    delay = DELAYS[difficulty]
    paused = False

    canvas.delete("all")
    canvas.delete("snake")
    canvas.delete(food)

    food_x = random.randint(0, (WIDTH - SIZE) // SIZE) * SIZE
    food_y = random.randint(0, (HEIGHT - SIZE) // SIZE) * SIZE
    food = canvas.create_oval(
        MARGIN + food_x,
        MARGIN + food_y,
        MARGIN + food_x + SIZE,
        MARGIN + food_y + SIZE,
        fill="red",
    )

    score_label.config(text="Score: {}".format(score))
    update_score_history()

    move()


start_button = tk.Button(window, text="Start Game", command=start_game)
start_button.pack()

restart_button = tk.Button(window, text="Restart Game", command=restart_game)
restart_button.pack()

window.mainloop()
